In [1]:
import sqlite3
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tqdm import tqdm
from DeepFM import *

c:\Users\cathy\.virtualenvs\content-recommendation-mCG3KVaj\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\cathy\.virtualenvs\content-recommendation-mCG3KVaj\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### recommendation preprocessing

In [61]:
# read db file
db_path = 'Retrieval.db'
conn = sqlite3.connect(db_path)


In [62]:
# print tables in the database
def print_tables(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table in tables:
        print(table[0])
    cursor.close()
    
print_tables(conn)

recommendations


In [63]:
# get the table and save it as a dataframe
def get_table_as_dataframe(conn, table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    return df

results_df = get_table_as_dataframe(conn, 'recommendations')
results_df

,model,user_id,business_id,real_label
0,UserCF,--4AjktZiHowEIBCMd4CZA,eXKblEHP3YJYU1Awz08hVw,0
1,UserCF,--4AjktZiHowEIBCMd4CZA,vuE1iseFrgNPumUEfHIZZQ,0
2,UserCF,--4AjktZiHowEIBCMd4CZA,EtKSTHV5Qx_Q7Aur9o4kQQ,0
3,UserCF,--4AjktZiHowEIBCMd4CZA,kZ1q0K13tFYG_ZJrVvsJHA,0
4,UserCF,--4AjktZiHowEIBCMd4CZA,R17gwW6zn9ilslbdvKdgsg,0
...,...,...,...,...
5559963,DSSM,0B77x9T49CeIBPMdLzwAHQ,Kh5SojHNsEU-4CLemEQnXA,0
5559964,DSSM,0B77x9T49CeIBPMdLzwAHQ,mU_iRT-NtND82T6xar7UxQ,0
5559965,DSSM,0B77x9T49CeIBPMdLzwAHQ,xJ3aziMabFEjF8X0kvPfLw,0
5559966,DSSM,0B77x9T49CeIBPMdLzwAHQ,3qRDGcBOcfFnEfAzVLO6oA,0


In [64]:
results_df['real_label'].value_counts()

real_label
0    5559229
1        739
Name: count, dtype: int64

In [ ]:
# Step 1: Count how many unique models each user_id has
user_model_counts = results_df.groupby('user_id')['model'].nunique()

# Step 2: Filter user_ids that appear in all 3 models
valid_users = user_model_counts[user_model_counts == 3].index

# Step 3: Filter the original DataFrame to keep only those users
filtered_df = results_df[results_df['user_id'].isin(valid_users)]


In [25]:
filtered_df

,model,user_id,business_id,real_label
0,UserCF,--4AjktZiHowEIBCMd4CZA,eXKblEHP3YJYU1Awz08hVw,0
1,UserCF,--4AjktZiHowEIBCMd4CZA,vuE1iseFrgNPumUEfHIZZQ,0
2,UserCF,--4AjktZiHowEIBCMd4CZA,EtKSTHV5Qx_Q7Aur9o4kQQ,0
3,UserCF,--4AjktZiHowEIBCMd4CZA,kZ1q0K13tFYG_ZJrVvsJHA,0
4,UserCF,--4AjktZiHowEIBCMd4CZA,R17gwW6zn9ilslbdvKdgsg,0
...,...,...,...,...
5414963,DSSM,08wIbJCYDZoYdbkDAMAb3w,0zWYKUPvImD7ufl1RQ1jhw,0
5414964,DSSM,08wIbJCYDZoYdbkDAMAb3w,P0-DrcqRjYEVqWBf84WPZw,0
5414965,DSSM,08wIbJCYDZoYdbkDAMAb3w,ZrJql3kVW--4hgDQBlA3NQ,0
5414966,DSSM,08wIbJCYDZoYdbkDAMAb3w,0K4RwxdAcViifyU3Htzxww,0


In [29]:
# count unique user_ids in the filtered DataFrame
unique_user_count = filtered_df['user_id'].nunique()
print(f"Number of unique user_ids with all 3 models: {unique_user_count}")

Number of unique user_ids with all 3 models: 527


In [30]:
# output the filtered DataFrame to a CSV file
output_csv_path = 'recommendation.csv'
filtered_df.to_csv(output_csv_path, index=False)

### function

In [2]:
def retrieve_user_info(user_ids):
    # Path to the user database
    db_path_user = '../../data/processed_data/yelp_data/yelp_user_data.db'

    # Connect to the database
    conn_user = sqlite3.connect(db_path_user)
    user_info = {}

    # Convert user_ids to a list if it's a single string
    if type(user_ids) == str:
        user_ids = [user_ids]

    try:
        # Fetch user details including the new fields
        user_details_query = f"""
        SELECT user_id, name, review_count, yelping_since, useful, funny, cool, fans,
               average_stars, friends, elite, compliment_hot, compliment_more,
               compliment_profile, compliment_cute, compliment_list, compliment_note,
               compliment_plain, compliment_cool, compliment_funny,
               compliment_writer, compliment_photos
        FROM user_data
        WHERE user_id IN ({','.join(['?' for _ in user_ids])})
        """
        cursor = conn_user.execute(user_details_query, user_ids)
        user_details = cursor.fetchall()

        # Add user details to result
        for user in user_details:
            user_id = user[0]
            user_info[user_id] = {
                "user_id": user_id,
                "name": user[1],
                "review_count": user[2],
                "yelping_since": user[3],
                "useful": user[4],
                "funny": user[5],
                "cool": user[6],
                "fans": user[7],
                "average_stars": user[8],
                "friends": [],
                "elite": [],
                "compliments": {
                    "hot": user[11],
                    "more": user[12],
                    "profile": user[13],
                    "cute": user[14],
                    "list": user[15],
                    "note": user[16],
                    "plain": user[17],
                    "cool": user[18],
                    "funny": user[19],
                    "writer": user[20],
                    "photos": user[21],
                }
            }
            if user[9]:
                user_info[user_id]["friends"] = user[9].split(',')
            if user[10]:
                user_info[user_id]["elite"] = user[10].split(',')

    except Exception as e:
        print(f"Error retrieving user info: {str(e)}")

    finally:
        # Close database connection
        conn_user.close()

    return user_info

In [34]:
def retrieve_business_info(business_ids):
    # Paths to database files
    db_path_business = '../../data/processed_data/yelp_data/yelp_business_data.db'
    db_path_review = '../../data/processed_data/yelp_data/yelp_review_data.db'
    db_path_user = '../../data/processed_data/yelp_data/yelp_user_data.db'
    db_path_tip = '../../data/processed_data/yelp_data/yelp_tip_data.db'

    # Connect to databases
    conn_business = sqlite3.connect(db_path_business)
    conn_review = sqlite3.connect(db_path_review)
    conn_user = sqlite3.connect(db_path_user)
    conn_tip = sqlite3.connect(db_path_tip)

    business_info = {}

    try:
        # First, initialize all with default values
        for business_id in business_ids:
            business_info[business_id] = {
                "business_id": business_id,
                "name": None,
                "address": None,
                "city": None,
                "state": None,
                "postal_code": None,
                "latitude": 0.0,
                "longitude": 0.0,
                "stars": 0.0,
                "review_count": 0,
                "is_open": 0,
                "attributes": None,
                "hours": None,
                "categories": [],
                "reviews": [],
                "tips": [],
                "checkins": []
            }
        
        # Fetch business details including the new fields: categories, attributes, hours
        business_details_query = f"""
        SELECT b.business_id, b.name, b.address, b.city, b.state, b.postal_code, b.latitude, b.longitude, 
               b.stars, b.review_count, b.is_open, b.attributes, b.hours
        FROM business_details b
        WHERE b.business_id IN ({','.join(['?' for _ in business_ids])})
        """
        cursor = conn_business.execute(business_details_query, business_ids)
        business_details = cursor.fetchall()

        # Add business details to result, including attributes and hours
        for business in business_details:
            business_id = business[0]
            business_info[business_id].update({
                "business_id": business_id,
                "name": business[1],
                "address": business[2],
                "city": business[3],
                "state": business[4],
                "postal_code": business[5],
                "latitude": business[6],
                "longitude": business[7],
                "stars": business[8],
                "review_count": business[9],
                "is_open": business[10],
                "attributes": business[11],
                "hours": business[12],
                "categories": [],  # Placeholder for categories
                "reviews": [],
                "tips": [],
                "checkins": []
            })

        # Fetch categories for each business
        category_query = f"""
        SELECT business_id, category
        FROM business_categories
        WHERE business_id IN ({','.join(['?' for _ in business_ids])})
        """
        cursor = conn_business.execute(category_query, business_ids)
        categories = cursor.fetchall()

        # Add categories to corresponding businesses
        for category in categories:
            business_id = category[0]
            if business_id in business_info:
                business_info[business_id]['categories'].append(category[1])

        # Fetch reviews
        review_query = f"SELECT * FROM review_data WHERE business_id IN ({','.join(['?' for _ in business_ids])})"
        cursor = conn_review.execute(review_query, business_ids)
        reviews = cursor.fetchall()

        # Add reviews to corresponding businesses
        for review in reviews:
            business_id = review[2]
            review_data = {
                "review_id": review[0],
                "user_id": review[1],
                "stars": review[3],
                "date": review[4],
                "text": review[5],
                "useful": review[6],
                "funny": review[7],
                "cool": review[8]
            }
            if business_id in business_info:
                business_info[business_id]['reviews'].append(review_data)

        # Fetch tips
        tip_query = f"SELECT * FROM tip_data WHERE business_id IN ({','.join(['?' for _ in business_ids])})"
        cursor = conn_tip.execute(tip_query, business_ids)
        tips = cursor.fetchall()

        # Add tips to corresponding businesses
        for tip in tips:
            business_id = tip[1]
            tip_data = {
                "user_id": tip[0],
                "text": tip[2],
                "date": tip[3],
                "compliment_count": tip[4]
            }
            if business_id in business_info:
                business_info[business_id]['tips'].append(tip_data)

        # Fetch check-ins
        checkin_query = f"SELECT * FROM checkin_data WHERE business_id IN ({','.join(['?' for _ in business_ids])})"
        cursor = conn_business.execute(checkin_query, business_ids)
        checkins = cursor.fetchall()

        # Add check-in data to corresponding businesses
        for checkin in checkins:
            business_id = checkin[0]
            checkin_data = {
                "checkin_date": checkin[1]
            }
            if business_id in business_info:
                business_info[business_id]['checkins'].append(checkin_data)

    except Exception as e:
        print(f"Error retrieving business info: {str(e)}")

    finally:
        # Close database connections
        conn_business.close()
        conn_review.close()
        conn_user.close()
        conn_tip.close()

    return business_info

### DeepFM

In [4]:
df_candidates = pd.read_csv("recommendation.csv")

In [5]:
all_ranked_results = []

### debug

In [6]:
df_candidates['user_id'].unique()

array(['--4AjktZiHowEIBCMd4CZA', '--KsuCSkGGvDKTbdK9NvIg',
       '--Vu3Gux9nPnLcG9yO_HxA', '--XwFm4qERD6J5SX0JAsbg',
       '--_H9j6ggxvqhh9nPofZwg', '--_r6E98SNIrGU7weyNxbw',
       '--u09WAjW741FdfkJXxNmg', '-02tzlpQosZ9HE85jEtV6Q',
       '-05wJ5BVkv8lYBIC_H5F-Q', '-0H6Rm6dCi3pkFBC26HsoQ',
       '-0KrCHEsOcjJ6N4k_k1A9A', '-0rW5xrSCH1HCnSEmoQA5A',
       '-1-pTXLHph2vcqRCNeT4_Q', '-13RX4Gy_F-zoLIenWAo-w',
       '-16bjdAbpdPLuGrfGPSPNQ', '-1MF2tosrw2WcCxeVNk81Q',
       '-1MiSauypbVtNnWts4aXpA', '-1WbN1Qd-opw8u3uEqs2Kg',
       '-2cKJFFNJ9XVyWBt62mWvA', '-2nNiAnDaSbQayynsqgL6Q',
       '-2zeaby7CmyAb6AIjX0cmw', '-3HYmxW_5Gsg4I0eH3ju-Q',
       '-3agoL-p87vZteiDzrz5og', '-3m3CcApRSL_k2XSluzuMg',
       '-3s52C4zL_DHRK0ULG6qtg', '-43uAiZZ6wsGhaurBi3Cdw',
       '-4D4Q_Fxj_rEL32t0VtzFQ', '-4RMTrzXEjs0ttG7stsl2Q',
       '-4adygW78x5K_xSG0u3Y6Q', '-4mjLXqAoAtQjqDFo9q9WQ',
       '-4r6OoklyemH7Up70wIhMA', '-50lprJ4kZ-XCM1hPU2vqg',
       '-584avr5GMIqv-9Ut85Ywg', '-5Slg5hO3useiypXYgZkGg

In [42]:
user_id = '--4AjktZiHowEIBCMd4CZA'
user_info = retrieve_user_info(user_id)
user_df = df_candidates[df_candidates['user_id'] == user_id]
business_ids = user_df['business_id'].tolist()
business_ids = list(set(business_ids))
business_info = retrieve_business_info(business_ids)

In [43]:
len(business_info.keys())

5598

In [44]:
len(business_ids)

5598

In [46]:
encoded_user_id = DeepFM_user_id_encoder.transform([user_id])[0]
user_continuous_features = ['review_count', 'useful', 'funny', 'cool', 'fans', 'average_stars']
user_compliments = ['hot', 'more', 'profile', 'cute', 'list', 'note', 'plain', 'cool', 'funny', 'writer', 'photos']

user_data_list = []

for user_id in user_info:
    # Extract continuous features
    user_features = [user_info[user_id].get(feat, 0) for feat in user_continuous_features]
    
    # One-hot encode compliments
    user_compliments_vector = [1 if c in user_info[user_id].get('compliments', []) else 0 for c in user_compliments]
    
    # Combine into a single row
    user_data_list.append(user_features + user_compliments_vector)

# Convert to DataFrame
user_continuous_features = pd.DataFrame(user_data_list, 
                                        columns=user_continuous_features + [f'compliment_{c}' for c in user_compliments])

# Scale continuous features
user_continuous_features = pd.DataFrame(DeepFM_user_scaler.transform(user_continuous_features), 
                                        columns=user_continuous_features.columns)

In [47]:
encoded_business_ids = DeepFM_business_id_encoder.transform(business_ids)

# later_features = ["name", "address", "city", "state", "postal_code",]
business_continuous_features = ["latitude", "longitude", "stars", 
"review_count"]

business_data_list = []
for business_id in business_info:
    # Extract continuous features
    business_features = [business_info[business_id].get(feat, 0) for feat in business_continuous_features]

    total_reviews = 0
    for review in business_info[business_id]['reviews']:
        total_reviews += review['stars']

    # Calculate average review
    if len(business_info[business_id]['reviews']) > 0:
        avg_review = total_reviews / len(business_info[business_id]['reviews'])
    else:
        avg_review = 0
    # Combine into a single row
    business_data_list.append(business_features + [avg_review])

business_continuous_features += ["avg_review"]

# Convert to DataFrame
business_continuous_features = pd.DataFrame(business_data_list, 
                                            columns=business_continuous_features)

ordered_business_continuous_features = ["stars", "review_count", "avg_review", "latitude", "longitude",] 

business_continuous_features = business_continuous_features[ordered_business_continuous_features]

# Scale continuous features and reorder columns
business_continuous_features = pd.DataFrame(DeepFM_business_scaler.transform(business_continuous_features), 
                                            columns=ordered_business_continuous_features)

In [48]:
user_features = np.repeat(user_continuous_features.values, len(business_continuous_features), axis=0)
business_features = np.tile(business_continuous_features.values, (len(user_continuous_features), 1))
all_features = np.concatenate([user_features, business_features], axis=1)

# get the user_ids and business_ids in the right format (i.e. -1, 1)
user_ids = np.repeat(encoded_user_id, len(encoded_business_ids)).reshape(-1, 1)
business_ids = np.array(encoded_business_ids).reshape(-1, 1)

In [49]:
# check shape of user_ids and business_ids
print(user_ids.shape)
print(business_ids.shape)
print(all_features.shape)

(5598, 1)
(5598, 1)
(5598, 22)


In [50]:
user_features.shape, business_features.shape, all_features.shape

((5598, 17), (5598, 5), (5598, 22))

In [52]:
predictions = DeepFM_model.predict([all_features, user_ids, business_ids])

175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [53]:
predictions = [float(prediction) for prediction in predictions]

# decode the business_ids
decoded_business_ids = DeepFM_business_id_encoder.inverse_transform(encoded_business_ids)

# combine the business_ids and predictions and sort by predictions
recommended_businesses = list(zip(decoded_business_ids, predictions))
recommended_businesses.sort(key=lambda x: x[1], reverse=True)

C:\Users\cathy\AppData\Local\Temp\ipykernel_24932\2334907030.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions = [float(prediction) for prediction in predictions]


In [54]:
recommended_businesses

[('ytynqOUb3hjKeJfRj5Tshw', 5.222929954528809),
 ('-fs09akgCKv5rTTy7iUHUg', 5.119536876678467),
 ('TE2IEDNV0RcI6s1wTOP4fg', 5.1116180419921875),
 ('KTGEa8V9ec0QBYM_uk1YLQ', 5.1063032150268555),
 ('ctHjyadbDQAtUFfkcAFEHw', 5.100107192993164),
 ('o_N7HiU3f6cVvrJAFrE43A', 5.092024803161621),
 ('zLm2PaYrpHkcr63ekwvtBQ', 5.084012985229492),
 ('5tK4y67IUn3eDd3S1bka9Q', 5.049875259399414),
 ('XJvOamdhTQI65jmzTfreDQ', 4.9942731857299805),
 ('NnlQhXtuVOVjUFYB1Hqp8Q', 4.992379665374756),
 ('Bk6uzC83wbmCHm2i5Uz8OA', 4.98884916305542),
 ('_ZMf5Am_e0AE7aveELJd9w', 4.986114501953125),
 ('8j5bQ0nDwoGiFaTQDNCQTg', 4.985906600952148),
 ('9pwzHmhCpgJwfaIP9jWoCg', 4.982203483581543),
 ('wUnLSg_GKfEIQ5CQQ770_g', 4.978819847106934),
 ('_PQzeo8P2sGczzFEifrK-Q', 4.972320556640625),
 ('atZ_olNKXOG4rEr6mccN8g', 4.968672752380371),
 ('tBQcVd9c0EHKAXLZn_v54g', 4.967219352722168),
 ('X3NS8516o015Z1zcOwrqmQ', 4.958934307098389),
 ('cOaw7LOj7yjCH9ty8eIJDg', 4.957681655883789),
 ('Y6K9DT44-4Md-ycMkC0nLg', 4.95752954

### batch retrieval

In [55]:
# Use tqdm for a progress bar
for user_id in tqdm(df_candidates['user_id'].unique()):

    # Subset data for this user
    user_df = df_candidates[df_candidates['user_id'] == user_id]

    # Get business_ids for the user
    business_ids = user_df['business_id'].tolist()
    business_ids = list(set(business_ids))
    
    user_info = retrieve_user_info(user_id)
    business_info = retrieve_business_info(business_ids)
    
    # Call your DeepFM rank function (assumes it returns sorted list of (business_id, score))
    try:
        ranked_results = DeepFM_rank_top_k(user_id, business_ids, user_info, business_info, k=1000)
        # top_k=None means keep full sorted list; or set top_k=100 if you want only top 100
    except Exception as e:
        print(f"Error ranking user {user_id}: {e}")
        continue

    # Turn into DataFrame for merging
    ranked_df = pd.DataFrame(ranked_results, columns=['business_id', 'score'])
    ranked_df['user_id'] = user_id

    # Optional: Add original label for evaluation later
    ranked_df = ranked_df.merge(user_df[['business_id', 'real_label']], on='business_id', how='left')

    all_ranked_results.append(ranked_df)


  0%|          | 0/527 [00:00<?, ?it/s]

175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


  0%|          | 1/527 [00:03<28:39,  3.27s/it]

174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


  0%|          | 2/527 [00:06<30:13,  3.45s/it]

...

100%|█████████▉| 525/527 [32:22<00:06,  3.36s/it]

172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


100%|█████████▉| 526/527 [32:26<00:03,  3.41s/it]

172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


100%|██████████| 527/527 [32:29<00:00,  3.70s/it]


### save the result

In [57]:
all_ranked_results

[                 business_id     score                 user_id  real_label
 0     ytynqOUb3hjKeJfRj5Tshw  5.222930  --4AjktZiHowEIBCMd4CZA           0
 1     -fs09akgCKv5rTTy7iUHUg  5.119537  --4AjktZiHowEIBCMd4CZA           0
 2     TE2IEDNV0RcI6s1wTOP4fg  5.111618  --4AjktZiHowEIBCMd4CZA           0
 3     KTGEa8V9ec0QBYM_uk1YLQ  5.106303  --4AjktZiHowEIBCMd4CZA           0
 4     ctHjyadbDQAtUFfkcAFEHw  5.100107  --4AjktZiHowEIBCMd4CZA           0
 ...                      ...       ...                     ...         ...
 1090  jcnKA1gG7Tek_UT2zWBI6w  4.576748  --4AjktZiHowEIBCMd4CZA           0
 1091  t5T6NYFsOqnYl0NETDgj7g  4.576547  --4AjktZiHowEIBCMd4CZA           0
 1092  IvAa7PvfOYl0xRML5W3ebg  4.576479  --4AjktZiHowEIBCMd4CZA           0
 1093  IvAa7PvfOYl0xRML5W3ebg  4.576479  --4AjktZiHowEIBCMd4CZA           0
 1094  Gt_MCSfwCSUDDhncJlxx3g  4.576457  --4AjktZiHowEIBCMd4CZA           0
 
 [1095 rows x 4 columns],
                  business_id     score                 user

In [58]:
df_all_ranked = pd.concat(all_ranked_results, ignore_index=True)

In [59]:
df_all_ranked

,business_id,score,user_id,real_label
0,ytynqOUb3hjKeJfRj5Tshw,5.222930,--4AjktZiHowEIBCMd4CZA,0
1,-fs09akgCKv5rTTy7iUHUg,5.119537,--4AjktZiHowEIBCMd4CZA,0
2,TE2IEDNV0RcI6s1wTOP4fg,5.111618,--4AjktZiHowEIBCMd4CZA,0
3,KTGEa8V9ec0QBYM_uk1YLQ,5.106303,--4AjktZiHowEIBCMd4CZA,0
4,ctHjyadbDQAtUFfkcAFEHw,5.100107,--4AjktZiHowEIBCMd4CZA,0
...,...,...,...,...
554952,VJ2BIGD9Q-vyE960n-4-dA,4.301839,08wIbJCYDZoYdbkDAMAb3w,0
554953,kvb0a9FiFs9UXWxE6q-5DA,4.301810,08wIbJCYDZoYdbkDAMAb3w,0
554954,DXNKGOCVmFP5iU8owhK2Fw,4.301074,08wIbJCYDZoYdbkDAMAb3w,0
554955,aROWvF8ltWk6LLNO62haag,4.300865,08wIbJCYDZoYdbkDAMAb3w,0


In [60]:
df_all_ranked.to_csv('all_ranked_results.csv', index=False)